In [45]:
import requests
import csv
from dotenv import load_dotenv
import os
import googlemaps
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Limpens\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Limpens\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [46]:
load_dotenv()
API_KEY = os.getenv("MAPS_API_KEY")

gmaps = googlemaps.Client(key=API_KEY)

viana_axis_hotel = 'AXIS VIANA BUSINESS & SPA HOTEL'
ponte_lima_axis_hotel = 'Axis Ponte de Lima Golf Resort Hotel'
ofir_axis_hotel = 'Axis Ofir Beach Resort Hotel'
braga_axis_hotel = 'Basic Braga by Axis'
vermar_axis_hotel = 'Hotel Axis Vermar Conference & Beach Hotel'
porto_axis_hotel = 'Axis Porto Business & SPA Hotel'
porto_club_axis_hotel = 'Axis Porto Club Hotel'

def get_hotel_reviews(hotel_name, api_key):
    """Fetches all reviews for a hotel using the Google Places API, handling pagination."""
    place_id = gmaps.find_place(input=hotel_name, input_type='textquery', fields=['place_id'])['candidates'][0]['place_id']
    all_reviews = []
    next_page_token = None

    while True:
        place_details = gmaps.place(place_id, fields=['name', 'reviews'])
        reviews = place_details['result']['reviews']
        all_reviews.extend(reviews)

        if 'next_page_token' in place_details['result']:
            next_page_token = place_details['result']['next_page_token']
        else:
            break

    return all_reviews

def save_reviews_to_csv(reviews, file_name):
    """Saves hotel reviews to a CSV file."""
    with open(file_name, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Review']) 
        for review in reviews:
            writer.writerow([review['text']]) 
            
viana_reviews = get_hotel_reviews(viana_axis_hotel, API_KEY)
save_reviews_to_csv(viana_reviews, 'viana_reviews.csv')

ponte_lima_reviews = get_hotel_reviews(ponte_lima_axis_hotel, API_KEY)
save_reviews_to_csv(ponte_lima_reviews, 'ponte_lima_reviews.csv')

ofir_reviews = get_hotel_reviews(ofir_axis_hotel, API_KEY)
save_reviews_to_csv(ofir_reviews, 'ofir_reviews.csv')

braga_reviews = get_hotel_reviews(braga_axis_hotel, API_KEY)
save_reviews_to_csv(braga_reviews, 'braga_reviews.csv')

vermar_reviews = get_hotel_reviews(vermar_axis_hotel, API_KEY)
save_reviews_to_csv(vermar_reviews, 'vermar_reviews.csv')

porto_business_reviews = get_hotel_reviews(porto_axis_hotel, API_KEY)
save_reviews_to_csv(porto_business_reviews, 'porto_business_reviews.csv')

porto_club_reviews = get_hotel_reviews(porto_club_axis_hotel, API_KEY)
save_reviews_to_csv(porto_club_reviews, 'porto_club_reviews.csv')

In [47]:
file_paths = ['viana_reviews.csv', 'ponte_lima_reviews.csv', 'ofir_reviews.csv', 
              'braga_reviews.csv', 'vermar_reviews.csv', 'porto_business_reviews.csv', 'porto_club_reviews.csv']

combined_df = pd.read_csv(file_paths[0])

for file_path in file_paths[1:]:
    temp_df = pd.read_csv(file_path, header=0) 
    combined_df = pd.concat([combined_df, temp_df], ignore_index=True)

combined_df.to_csv('combined.csv', index=False)

for file_path in file_paths:
    try:
        os.remove(file_path)
        print(f'{file_path} foi excluído com sucesso.')
    except Exception as e:
        print(f'Erro ao excluir {file_path}: {e}')

viana_reviews.csv foi excluído com sucesso.
ponte_lima_reviews.csv foi excluído com sucesso.
ofir_reviews.csv foi excluído com sucesso.
braga_reviews.csv foi excluído com sucesso.
vermar_reviews.csv foi excluído com sucesso.
porto_business_reviews.csv foi excluído com sucesso.
porto_club_reviews.csv foi excluído com sucesso.


In [48]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess_text(text):
    if not isinstance(text, str):
        return ''
    text = text.lower() 
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) 
    text = re.sub(r'\W', ' ', text)  
    text = re.sub(r'\d+', '', text)  
    words = word_tokenize(text) 
    words = [word for word in words if word not in stop_words] 
    words = [stemmer.stem(word) for word in words]  
    return ' '.join(words)

file_path = 'Combined.csv'
df = pd.read_csv(file_path)

if 'Review' in df.columns:
    df['Review'] = df['Review'].apply(preprocess_text)

    df.to_excel('Projeto_Processed.xlsx', index=False)
    print('Processamento concluído. O arquivo foi salvo como Projeto_Processed.xlsx.')
    
    try:
        os.remove(file_path)
        print(f'{file_path} foi excluído com sucesso.')
    except Exception as e:
        print(f'Erro ao excluir {file_path}: {e}')
else:
    print('A coluna "Review" não foi encontrada no arquivo.')

Processamento concluído. O arquivo foi salvo como Projeto_Processed.xlsx.
Combined.csv foi excluído com sucesso.


In [49]:
import pandas as pd
import os
from transformers import pipeline

file_path = 'Projeto_processed.xlsx' 
df = pd.read_excel(file_path)

coluna_review = 'Review'

print("\nCarregando o modelo de análise de sentimentos BERT...")
sentiment_pipeline = pipeline('sentiment-analysis')

print("\nAplicando a análise de sentimentos...")
df['Sentimento'] = df[coluna_review].apply(
    lambda x: sentiment_pipeline(x)[0]['label'] if isinstance(x, str) and pd.notnull(x) else 'Sem análise'
)

output_path = 'Projeto_with_sentiment.xlsx'
df.to_excel(output_path, index=False)
print(f"\nArquivo atualizado salvo com sucesso em: {output_path}")

try:
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"Arquivo original '{file_path}' deletado com sucesso.")
    else:
        print(f"O arquivo '{file_path}' não foi encontrado para exclusão.")
except Exception as e:
    print(f"Erro ao tentar deletar o arquivo '{file_path}': {e}")



No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.



Carregando o modelo de análise de sentimentos BERT...


Device set to use cpu



Aplicando a análise de sentimentos...

Arquivo atualizado salvo com sucesso em: Projeto_with_sentiment.xlsx
Arquivo original 'Projeto_processed.xlsx' deletado com sucesso.


In [50]:
df['Sentimento_Binario'] = df['Sentimento'].map({'POSITIVE': 1, 'NEGATIVE': 0})

output_path = 'Projeto_with_sentiment.xlsx'
df.to_excel(output_path, index=False)